In [1]:
# Use one of the recent language models (e.g., GPT) to generate more convincing Shakespearean text.
# The simplest way to use recent language models is to use the excellent transformers library, 
# open sourced by Hugging Face. It provides many modern neural net architectures 
# (including BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet and more) for Natural Language Processing (NLP), 
# including many pretrained models. It relies on either TensorFlow or PyTorch. Best of all: 
# it's amazingly simple to use.

In [2]:
# imports
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


import numpy as np

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.experimental.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")

print(keras.__version__)

from datetime import date

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.
2.4.0


In [3]:
# let's load a pretrained model. In this example, we will use OpenAI's GPT model, 
# with an additional Language Model on top (just a linear layer with weights tied to the input embeddings).

In [4]:
from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

All model checkpoint layers were used when initializing TFOpenAIGPTLMHeadModel.

All the layers of TFOpenAIGPTLMHeadModel were initialized from the model checkpoint at openai-gpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predictions without further training.


In [5]:
# We will also need a specialized tokenizer for this model. This one will try to use the spaCy and ftfy libraries 
# if they are installed, or else it will fall back to BERT's BasicTokenizer followed by Byte-Pair Encoding 
# (which should be fine for most use cases).

In [6]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [7]:
# tokenize the prmompt text
prompt_text = "This royal throne of kings, this sceptred isle"
encoded_prompt = tokenizer.encode(prompt_text,
                                  add_special_tokens=False,
                                  return_tensors="tf")
encoded_prompt

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187]], dtype=int32)>

In [8]:
# use the model to generate text after the prompt. We will generate 5 different sentences, 
# each starting with the prompt text, followed by 40 additional tokens. 
num_sequences = 5
length = 40

generated_sequences = model.generate(
    input_ids=encoded_prompt,
    do_sample=True,
    max_length=length + len(encoded_prompt[0]),
    temperature=1.0,
    top_k=0,
    top_p=0.9,
    repetition_penalty=1.0,
    num_return_sequences=num_sequences,
)

generated_sequences

<tf.Tensor: shape=(5, 50), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   488,   481,  7795,   498,  7144,   557,   239, 40477,
          674,   481, 10891,   260,  1092,  2923,  2034,   481,  1705,
         1294,  2075,   498,   481,  4204,   240,   481,  2998,  2360,
          525,  6851,  3415,   617,   987, 13703,  2034,   481,  2761,
          260,  5693,   240,   481,  5616],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   544,   808,   246,  1938,   638,   841,   617,   481,
        20090,   239,   568,   669,   481,  1621,   544,  2923,   606,
          994,   727,   551,   498,   481, 16187,   781,   775,  4000,
          485,  1529,   507,   504,   481,  4802,   239,   481, 29401,
         1549,   544,  1949,   239,   244],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   240,   544, 25898,   702,  1272, 19175,   240,   488,
          557

In [9]:
# decode the generated sequences and print them
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("-" * 80)

this royal throne of kings, this sceptred isle and the arch of pelas. 
 then the jewel - light settled upon the cold dark edge of the plain, the pale lights that shone forth from its cliffs upon the sea - shore, the brilliant
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle is only a short way away from the mainland. but when the matter is settled we should get out of the isle before any attempt to use it on the tower. the driftwood wall is clear. "
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle, is revered by many peoples, and as such is not used in our family, only for the king himself. it is said that the ancient king of the seas got a duty to the seas,
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle of insanity, and this king and these mages, who